In [ ]:
!pip install -U ckiptagger[tf,gdown]

In [5]:
# import library
from ckiptagger import data_utils

#設定儲存路徑(存在與此ipynb相同的目錄下)
#注意: 此模型需2GB的儲存空間
path_to_store = './'


#我們可以使用提供的API從iis-ckip或gdrive-ckip進行下載
#這裡我們使用gdrive-ckip進行下載

#從iis-ckip下載
#data_utils.download_data_url("./")


#下載到path_to_store並解壓縮(預設下載的壓縮檔為data.zip)
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
1.88GB [00:15, 118MB/s] 


In [6]:
# import library
import numpy as np

char_token_list = np.load('./data/embedding_character/token_list.npy') #讀入character list
char_vector_list = np.load('./data/embedding_character/vector_list.npy') #讀入character 的向量list

print(f'Total: {len(char_token_list)} characters token and {len(char_vector_list)}')

Total: 13136 characters token and 13136


In [7]:
print(f'First character: {char_token_list[0]}')
print(f'First character vector: {char_vector_list[0]}')
print(f'Each character vector has {char_vector_list[0].shape} dimension')

First character: ,
First character vector: [ 9.06846821e-02 -3.91008481e-02 -4.38107513e-02 -2.78249998e-02
 -6.59082830e-03  9.82730538e-02 -3.30615602e-02 -3.32455598e-02
 -8.25216696e-02 -5.97619563e-02 -2.15497568e-01  1.21742591e-01
  8.44441280e-02  6.46698922e-02 -1.73088256e-02  5.60178608e-02
 -4.69524190e-02  3.83560695e-02  2.81602979e-01 -1.05296642e-01
  2.53045380e-01 -1.28228694e-01  4.38064672e-02  8.75911787e-02
  6.25642985e-02 -4.81812134e-02  4.59593609e-02  7.77227730e-02
  1.72905862e-01 -1.51137233e-01 -3.13500345e-01  1.51329324e-01
  3.79214510e-02 -3.17767225e-02 -6.73223007e-03 -1.10702842e-01
  1.52048752e-01  2.12142825e-01 -9.31895450e-02  1.12124726e-01
  1.25201389e-01  5.70141450e-02 -1.24505214e-01 -4.11491245e-02
 -7.26862326e-02 -1.22127883e-01 -1.16852924e-01  8.49076360e-02
 -2.08125204e-01  8.73672292e-02  4.89221141e-02  4.23294865e-02
 -1.74451604e-01 -7.16942502e-03 -1.21610258e-02 -3.41356136e-02
 -3.82989980e-02  1.12134412e-01  8.02825466e-0

In [8]:
word_token_list = np.load('./data/embedding_word/token_list.npy') #讀入character list
word_vector_list = np.load('./data/embedding_word/vector_list.npy') #讀入character 的向量list

print(f'Total: {len(word_token_list)} characters token and {len(word_vector_list)}')

Total: 1355791 characters token and 1355791


In [9]:
#使用CkipTagger進行斷詞
#因為CkipTagger是基於TensorFlow搭建的Cross-BiLSTM深度學習模型，因此在進行斷詞、詞性標註、命名實體識別時可以使用GPU加速操作。 
from ckiptagger import WS, POS, NER

ws = WS("./data") #建構斷詞
input_traditional_str = ['小明碩士畢業於國立臺灣大學，現在在日本東京大學進修深造', "… 你確定嗎… 不要再騙了……"]

word_sentence_list = ws(
    input_traditional_str,
    sentence_segmentation = True, # To consider delimiters
    segment_delimiter_set = {",", "。", ":", "?", "!", ";"}) # This is the defualt set of delimiters

print(word_sentence_list)

[['小明', '碩士', '畢業', '於', '國立', '臺灣', '大學', '，', '現在', '在', '日本', '東京', '大學', '進修', '深造'], ['…', ' ', '你', '確定', '嗎', '…', ' ', '不要', '再', '騙', '了', '…', '…']]


In [10]:
# 使用Ckiptagger進行詞性標註

pos = POS("./data")

pos_sentence_list = pos(word_sentence_list)
print(pos_sentence_list)

[['Nb', 'Na', 'VH', 'P', 'A', 'Nc', 'Nc', 'COMMACATEGORY', 'Nd', 'P', 'Nc', 'Nc', 'Nc', 'VC', 'VA'], ['ETCCATEGORY', 'WHITESPACE', 'Nh', 'VK', 'T', 'ETCCATEGORY', 'WHITESPACE', 'D', 'D', 'VC', 'Di', 'ETCCATEGORY', 'ETCCATEGORY']]


In [11]:
# 使用Ckiptagger進行命名實體辨識

ner = NER("./data")

entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
print(entity_sentence_list)

[{(17, 23, 'ORG', '日本東京大學'), (7, 13, 'ORG', '國立臺灣大學')}, set()]


In [ ]:
#上述返回的list中,數字表自字串在輸入文本中的位置,而第三個值代表屬於的命名實體類別。 
#舉例來說(17, 23, 'ORG', '日本東京大學'), 代表日本東京大學在文本中的位置起始為17,結束為23,而其所屬的命名實體類別為 ORG表機構的意思。

In [12]:
#帶入自定義字典
#如同結巴一樣,CkipTagger也提供使用者輸入自定義字典。
# import library
from ckiptagger import construct_dictionary

#定義自定義字典
#每個字對應的數值為其相對權重

word_to_weight = {
    "土地公": 1,
    "土地婆": 1,
    "公有": 2,
    "": 1,
    "來亂的": "啦",
    "國立臺灣大學": 1,
    "日本東京": 1
}
dictionary1 = construct_dictionary(word_to_weight)
print(dictionary1)

[(2, {'公有': 2.0}), (3, {'土地公': 1.0, '土地婆': 1.0}), (4, {'日本東京': 1.0}), (6, {'國立臺灣大學': 1.0})]


In [13]:
word_to_weight = {
    "日本東京大學": 1
}
dictionary2 = construct_dictionary(word_to_weight)
print(dictionary2)

[(6, {'日本東京大學': 1.0})]


In [14]:
ws = WS("./data") #建構斷詞
input_traditional_str = ['小明碩士畢業於國立臺灣大學，現在在日本東京大學進修深造', "… 你確定嗎… 不要再騙了……"]

word_sentence_list = ws(
    input_traditional_str,
    sentence_segmentation = True, # To consider delimiters
    segment_delimiter_set = {",", "。", ":", "?", "!", ";"}, # This is the defualt set of delimiters
    recommend_dictionary = dictionary1, # words in this dictionary are encouraged 建議使用字典
    coerce_dictionary = dictionary2) # words in this dictionary are forced 強制使用字典


print(word_sentence_list)

[['小明', '碩士', '畢業', '於', '國立臺灣大學', '，', '現在', '在', '日本東京大學', '進修', '深造'], ['…', ' ', '你', '確定', '嗎', '…', ' ', '不要', '再', '騙', '了', '…', '…']]


In [ ]:
#第一個字典中將日本東京定義為一個詞, 而第二個字典將日本東京大學定義為一個字, 在使用斷詞時,輸出的結果會按照強制使用字典的建議來斷詞。